# Tool Calling Use Cases

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

In [1]:
import warnings
warnings.filterwarnings('ignore')

## Get API keys

In [2]:
from utils import get_tavily_api_key
TAVILY_API_KEY = get_tavily_api_key()

## Load helper functions

In [3]:
from utils import llama31
from utils import cprint
import json

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>utils.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Define tool system prompt

In [4]:
from datetime import datetime

current_date = datetime.now()
formatted_date = current_date.strftime("%d %B %Y")
print(formatted_date)

10 October 2024


In [5]:
tool_system_prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""

## The brave_search built-in tool

In [19]:
prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the current weather in Tunisia, Sousse?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

In [20]:
response = llama31(prompt)
print(response)



brave_search.call(query="current weather in Tunisia Sousse")


In [21]:
no_tool_call_prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the population of California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

In [22]:
no_tool_call_response = llama31(no_tool_call_prompt)
print(no_tool_call_response)



As of my knowledge cutoff in 2023, the estimated population of California is approximately 39.5 million people, according to the United States Census Bureau. However, please note that this number may have changed since my knowledge cutoff date. For the most up-to-date information, I recommend checking the latest data from the United States Census Bureau or other reliable sources.


### Calling the search API

In [24]:
from tavily import TavilyClient
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

result = tavily_client.search("current weather in Tunisia, Sousse")
cprint(result)

{
    "query": "current weather in Tunisia, Sousse",
    "follow_up_questions": null,
    "answer": null,
    "images": [],
    "results": [
        {
            "title": "Weather in Sousse, Tunisia",
            "url": "https://www.weatherapi.com/",
            "content": "{'location': {'name': 'Sousse', 'region': 'Sousse', 'country': 'Tunisia', 'lat': 35.8256, 'lon': 10.6411, 'tz_id': 'Africa/Tunis', 'localtime_epoch': 1728568122, 'localtime': '2024-10-10 14:48'}, 'current': {'last_updated_epoch': 1728567900, 'last_updated': '2024-10-10 14:45', 'temp_c': 29.0, 'temp_f': 84.2, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 8.7, 'wind_kph': 14.0, 'wind_degree': 66, 'wind_dir': 'ENE', 'pressure_mb': 1009.0, 'pressure_in': 29.8, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 52, 'cloud': 0, 'feelslike_c': 29.4, 'feelslike_f': 84.9, 'windchill_c': 29.0, 'windchill_f': 84.3, 'heatindex_c': 29.4, 'heatindex_

### Reprompting Llama with search tool response

In [25]:
search_result = result["results"][0]["content"]
print(search_result)

{'location': {'name': 'Sousse', 'region': 'Sousse', 'country': 'Tunisia', 'lat': 35.8256, 'lon': 10.6411, 'tz_id': 'Africa/Tunis', 'localtime_epoch': 1728568122, 'localtime': '2024-10-10 14:48'}, 'current': {'last_updated_epoch': 1728567900, 'last_updated': '2024-10-10 14:45', 'temp_c': 29.0, 'temp_f': 84.2, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 8.7, 'wind_kph': 14.0, 'wind_degree': 66, 'wind_dir': 'ENE', 'pressure_mb': 1009.0, 'pressure_in': 29.8, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 52, 'cloud': 0, 'feelslike_c': 29.4, 'feelslike_f': 84.9, 'windchill_c': 29.0, 'windchill_f': 84.3, 'heatindex_c': 29.4, 'heatindex_f': 85.0, 'dewpoint_c': 15.7, 'dewpoint_f': 60.2, 'vis_km': 8.0, 'vis_miles': 4.0, 'uv': 3.8, 'gust_mph': 13.4, 'gust_kph': 21.5}}


In [28]:
prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>
What is the current weather in Tunisia, Sousse?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|python_tag|>{response}<|eom_id|>
<|start_header_id|>ipython<|end_header_id|>

{search_result}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 10 October 2024
<|eot_id|><|start_header_id|>user<|end_header_id|>
What is the current weather in Tunisia, Sousse?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|python_tag|>The API result does not contain the weather in Menlo Park, California.  Let me try searching again.<|eom_id|>
<|start_header_id|>ipython<|end_header_id|>

{'location': {'name': 'Sousse', 'region': 'Sousse', 'country': 'Tunisia', 'lat': 35.8256, 'lon': 10.6411, 'tz_id': 'Africa/Tunis', 'localtime_epoch': 1728568122, 'localtime': '2024-10-10 14:48'}, 'current': {'last_updated_epoch': 1728567900, 'last_updated': '2024-10-10 14:45', 'temp_c': 29.0, 'temp_f': 84.2, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 8.7, 'wind_kph': 14.0, 'wind_degree': 66, 'wind_dir'

In [29]:
response = llama31(prompt)
print(response)

The current weather in Sousse, Tunisia is:

- Temperature: 29°C (84.2°F)
- Condition: Sunny
- Humidity: 52%
- Wind: 14 km/h (8.7 mph) from ENE
- Pressure: 1009 mb (29.8 in)
- Precipitation: 0 mm (0 in)
- Feels like: 29.4°C (84.9°F)
- Windchill: 29.0°C (84.3°F)
- Heat index: 29.4°C (85.0°F)
- Dew point: 15.7°C (60.2°F)
- Visibility: 4 km (2.5 miles)
- UV index: 3.8
- Gust speed: 21.5 km/h (13.4 mph)


## Using the higher-level message

In [30]:
system_prompt_content = f"""
Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""

In [31]:
messages = [
    {"role": "system", "content":  system_prompt_content},
    {"role": "user",   "content": "What is the current weather in Tunisia, Sousse?"}
  ]

In [32]:
response = llama31(messages)
print(response)

brave_search.call(query="current weather in Sousse, Tunisia")


In [35]:
messages = [
    {"role": "system",     "content":  system_prompt_content},
    {"role": "user",       "content": "What is the current weather in Tunisia, Sousse?"},
    {"role": "assistant",  "content": response},
    {"role": "ipython",    "content": search_result}
  ]

In [36]:
response = llama31(messages)
print(response)

Based on the search results, the current weather in Sousse, Tunisia is:

- Temperature: 29°C (84.2°F)
- Condition: Sunny
- Humidity: 52%
- Wind: 14 km/h (8.7 mph) from ENE
- Pressure: 1009 mb (29.8 in)
- Precipitation: 0 mm (0 in)
- Feels like: 29.4°C (84.9°F)
- Windchill: 29.0°C (84.3°F)
- Heat index: 29.4°C (85.0°F)
- Dew point: 15.7°C (60.2°F)
- Visibility: 8 km (4.0 miles)
- UV index: 3.8
- Gust speed: 21.5 km/h (13.4 mph)


## The Wolfram Alpha tool

In [37]:
math_problem = "Can you help me solve this equation: x^3 - 2x^2 - x + 2 = 0?"

In [38]:
messages = [
    {"role": "system",  "content": system_prompt_content},
    {"role": "user",    "content": math_problem}
]

In [39]:
response = llama31(messages)
print(response)

wolfram_alpha.call(query="solve x^3 - 2x^2 - x + 2 = 0")


### Calling the Wolfram Alpha tool

In [40]:
from utils import wolfram_alpha
tool_result = wolfram_alpha("solve x^3 - 2x^2 - x + 2 = 0")
print(tool_result)

x = ± 1
x = 2


### Checking the result

In [41]:
from sympy import symbols, Eq, solve

x = symbols('x')                           # Define the variable
equation = Eq(x**3 - 2*x**2 - 1*x + 2, 0) # Define the equation
solution = solve(equation, x)              # Solve the equation

print(solution)

[-1, 1, 2]


### Reprompting Llama with Wolfram Alpha tool response

In [42]:
messages = [
    {"role": "system", "content": system_prompt_content},
    {"role": "user",      "content": math_problem},
    {"role": "assistant", "content": response},
    {"role": "ipython",   "content": tool_result}
]

In [43]:
response = llama31(messages)
print(response)

The solutions to the equation x^3 - 2x^2 - x + 2 = 0 are x = ±1 and x = 2.


## The code interpreter built-in tool

In [44]:
loan_question = (
    "How much is the monthly payment, total payment, "
    "and total interest paid for a 30 year mortgage of $1M "
    "at a fixed rate of 6% with a 20% down payment?"
)

In [45]:
messages = [
    {"role": "system",     "content": system_prompt_content},
    {"role": "user",       "content": loan_question},
  ]

In [46]:
response = llama31(messages)
print(response)

# Import necessary modules
import math

# Define variables
principal = 800000  # $1M - $200,000 (20% down payment)
annual_interest_rate = 0.06  # 6%
years = 30
months = years * 12

# Calculate monthly interest rate
monthly_interest_rate = annual_interest_rate / 12

# Calculate monthly payment
monthly_payment = principal * (monthly_interest_rate * (1 + monthly_interest_rate) ** months) / ((1 + monthly_interest_rate) ** months - 1)

# Calculate total payment
total_payment = monthly_payment * months

# Calculate total interest paid
total_interest_paid = total_payment - principal

print(f"Monthly payment: ${monthly_payment:.2f}")
print(f"Total payment: ${total_payment:.2f}")
print(f"Total interest paid: ${total_interest_paid:.2f}")


# Calculate loan payments

In [47]:
from utils import calculate_loan
monthly_payment, total_payment, total_interest_paid = calculate_loan(
    loan_amount = 1000000, 
    annual_interest_rate = 0.06, 
    loan_term = 30, 
    down_payment = 200000)

print(f"Monthly payment: ${(monthly_payment)}")
print(f"Total payment: ${(total_payment)}")
print(f"Total interest paid: ${(total_interest_paid)}")

Monthly payment: $4796
Total payment: $1726705
Total interest paid: $926705


## Generating the code in Java

In [48]:
messages = [
    {"role":    "system",     
     "content": system_prompt_content + "\nGenerate the code in Java."},
    {"role":    "user",       "content": loan_question},
  ]

In [49]:
response = llama31(messages)
print(response)

import java.util.Scanner;

public class MortgageCalculator {
    public static void main(String[] args) {
        double principal = 1000000; // $1M
        double downPayment = 0.2 * principal; // 20% down payment
        double loanAmount = principal - downPayment;
        double annualInterestRate = 0.06; // 6% interest rate
        int years = 30; // 30 year mortgage

        double monthlyInterestRate = annualInterestRate / 12;
        int numberOfPayments = years * 12;

        double monthlyPayment = loanAmount * monthlyInterestRate * Math.pow(1 + monthlyInterestRate, numberOfPayments) / (Math.pow(1 + monthlyInterestRate, numberOfPayments) - 1);

        double totalPayment = monthlyPayment * numberOfPayments;
        double totalInterestPaid = totalPayment - loanAmount;

        System.out.println("Monthly Payment: $" + String.format("%.2f", monthlyPayment));
        System.out.println("Total Payment: $" + String.format("%.2f", totalPayment));
        System.out.println("Total 

### Reprompting Llama with Code Interpreter tool response

In [50]:
code_interpreter_tool_response="""
Monthly payment: $4796
Total payment: $1726705
Total interest paid: $926705
"""

In [51]:
messages = [
    {"role": "system", "content":  system_prompt_content},
    {"role": "user",      "content": loan_question},
    {"role": "assistant", "content": response},
    {"role": "ipython",   "content": code_interpreter_tool_response}
  ]

In [52]:
response = llama31(messages)
print(response)

The monthly payment for a 30-year mortgage of $1M at a fixed rate of 6% with a 20% down payment is $4,796. The total payment over the life of the loan is $1,726,705, and the total interest paid is $926,705.


## Llama 3.1 custom tool calling

In [53]:
from utils import trending_songs, get_boiling_point

country = "US"
top_num = 5
top_songs = trending_songs(country, top_num)
print(f"Top {top_num} trending songs in {country}:")
print(top_songs)

Top 5 trending songs in US:
['Blinding Lights - The Weeknd', 'Levitating - Dua Lipa', 'Peaches - Justin Bieber', 'Save Your Tears - The Weeknd', 'Good 4 U - Olivia Rodrigo']


<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>trending_songs()</code> in <code>utils.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Prompt Llama 3.1 for custom tool call

In [54]:
user_prompt = """
Answer the user's question by using the following functions if needed.
If none of the functions can be used, please say so.
Functions (in JSON format):
{
    "type": "function", "function": {
        "name": "get_boiling_point",
        "description": "Get the boiling point of a liquid",
        "parameters": {
            "type": "object", "properties": [
                {"liquid_name": {"type": "object", "description": "name of the liquid"}},
                {"celsius": {"type": "object", "description": "whether to use celsius"}}
            ], "required": ["liquid_name"]
        }
    }
}
{
    "type": "function", "function": {
        "name": "trending_songs",
        "description": "Returns the trending songs on a Music site",
        "parameters": {
            "type": "object", "properties": [
                {"country": {"type": "object", "description": "country to return trending songs for"}},
                {"n": {"type": "object", "description": "The number of songs to return"}}
            ], "required": ["country"]
        }
    }
}

Question: Can you check the top 5 trending songs in US?
"""

In [55]:
messages = [
    {
      "role": "system", "content":  f"""
Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""},
    {"role": "user", "content": user_prompt}
  ]

In [56]:
result = llama31(messages,405)
print(result)

{"type": "function", "name": "trending_songs", "parameters": {"country": "US", "n": "5"}}


### Calling the custom tool

In [57]:
custom_tools = {"trending_songs": trending_songs,
                "get_boiling_point": get_boiling_point}

In [58]:
res = json.loads(result)
function_name = res['name']
parameters = list(res['parameters'].values())
function_name, parameters

('trending_songs', ['US', '5'])

In [59]:
tool_result = custom_tools[function_name](*parameters)
tool_result

['Blinding Lights - The Weeknd',
 'Levitating - Dua Lipa',
 'Peaches - Justin Bieber',
 'Save Your Tears - The Weeknd',
 'Good 4 U - Olivia Rodrigo']

### Reprompting Llama with custom tool call result

In [60]:
messages = [
    {
      "role": "system", "content":  f"""
Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""},
    {"role": "user", "content": user_prompt},
    {"role": "assistant", "content": result},
    {"role": "ipython", "content": ','.join(tool_result)}
  ]

In [61]:
response = llama31(messages, 70)
print(response)

The top 5 trending songs in the US are:

1. Blinding Lights - The Weeknd
2. Levitating - Dua Lipa
3. Peaches - Justin Bieber
4. Save Your Tears - The Weeknd
5. Good 4 U - Olivia Rodrigo
